<b>Overview:</b> SparkFit is looking to open another location for a certified StrongFirst personal training studio and wants to know what area of NYC they should consider based off of 1) high volume subway commuters and 2) one that is not in close proximity to another StrongFirst certified gym.

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
import seaborn as sns

## <b> Data used in this project reflects May 2021-Apr 2022 MTA turnstile data.</b>
* MTA turnstile data is from http://web.mta.info/developers/turnstile.html
* All data was processed and imported from SQL using SQLalchemy in notebook #1, getting the data.

In [2]:
mta_df = pd.read_pickle("mta_df.pkl")

In [3]:
mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,00:00:00,REGULAR,7575503,2588978
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,04:00:00,RECOVR AUD,7575515,2588982
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,08:00:00,REGULAR,7575523,2588989
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,12:00:00,REGULAR,7575557,2588996
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,16:00:00,REGULAR,7575630,2589008


In [4]:
mta_df.shape

(40049393, 11)

In [5]:
mta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40049393 entries, 0 to 11125368
Data columns (total 11 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   C/A       object
 1   UNIT      object
 2   SCP       object
 3   STATION   object
 4   LINENAME  object
 5   DIVISION  object
 6   DATE      object
 7   TIME      object
 8   DESC      object
 9   ENTRIES   int64 
 10  EXITS     int64 
dtypes: int64(2), object(9)
memory usage: 3.6+ GB


### Transforming date into datetype.

In [6]:
# Take the date and time fields into a single datetime column
mta_df["DATE_TIME"] = pd.to_datetime(mta_df.DATE + " " + mta_df.TIME, format="%m/%d/%Y %H:%M:%S")

mta_df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATE_TIME
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,00:00:00,REGULAR,7575503,2588978,2021-05-22 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,04:00:00,RECOVR AUD,7575515,2588982,2021-05-22 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,08:00:00,REGULAR,7575523,2588989,2021-05-22 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,12:00:00,REGULAR,7575557,2588996,2021-05-22 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,05/22/2021,16:00:00,REGULAR,7575630,2589008,2021-05-22 16:00:00


In [7]:
mta_df.dtypes

C/A                  object
UNIT                 object
SCP                  object
STATION              object
LINENAME             object
DIVISION             object
DATE                 object
TIME                 object
DESC                 object
ENTRIES               int64
EXITS                 int64
DATE_TIME    datetime64[ns]
dtype: object

### Assess for duplicate Entries & Exits
* Assessing for Entries first.

In [8]:
# verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(mta_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(20)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
948821,B024,R211,00-05-00,KINGS HWY,2021-05-01 16:00:00,14
4628944,N329,R201,00-03-02,WOODHAVEN BLVD,2021-05-08 09:00:00,14
4628942,N329,R201,00-03-02,WOODHAVEN BLVD,2021-05-08 01:00:00,14
948842,B024,R211,00-05-00,KINGS HWY,2021-05-05 04:00:00,14
948841,B024,R211,00-05-00,KINGS HWY,2021-05-05 00:00:00,14
948840,B024,R211,00-05-00,KINGS HWY,2021-05-04 20:00:00,14
948839,B024,R211,00-05-00,KINGS HWY,2021-05-04 16:00:00,14
948838,B024,R211,00-05-00,KINGS HWY,2021-05-04 12:00:00,14
948837,B024,R211,00-05-00,KINGS HWY,2021-05-04 08:00:00,14
948835,B024,R211,00-05-00,KINGS HWY,2021-05-04 00:00:00,14


* We have numerous cases of duplicate entries.
* Looks to be a incorrect AUD entry for column "DESC".  May be we should just select where DESC = Regular.

In [9]:
mta_df.DESC.value_counts()

REGULAR       39874207
RECOVR AUD      175186
Name: DESC, dtype: int64

* Let's get rid of all the duplicate entries.

In [10]:
mta_df.sort_values(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], 
                          inplace=True, ascending=False)
mta_df.drop_duplicates(subset=["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"], inplace=True)


In [11]:
# Sanity Check to verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(mta_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .ENTRIES.count()
 .reset_index()
 .sort_values("ENTRIES", ascending=False)).head(5)

,C/A,UNIT,SCP,STATION,DATE_TIME,ENTRIES
0,A002,R051,02-00-00,59 ST,2021-04-24 00:00:00,1
7416748,R141,R031,00-00-00,34 ST-PENN STA,2022-01-27 23:00:00,1
7416740,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 11:00:00,1
7416741,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 15:00:00,1
7416742,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 19:00:00,1


* Now assessing duplicates for Exits...

In [12]:
# verify that "C/A", "UNIT", "SCP", "STATION", "DATE_TIME" is unique
(mta_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])
 .EXITS.count()
 .reset_index().sort_values("EXITS", ascending=False)).head(20)

,C/A,UNIT,SCP,STATION,DATE_TIME,EXITS
0,A002,R051,02-00-00,59 ST,2021-04-24 00:00:00,1
7416748,R141,R031,00-00-00,34 ST-PENN STA,2022-01-27 23:00:00,1
7416740,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 11:00:00,1
7416741,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 15:00:00,1
7416742,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 19:00:00,1
7416743,R141,R031,00-00-00,34 ST-PENN STA,2022-01-26 23:00:00,1
7416744,R141,R031,00-00-00,34 ST-PENN STA,2022-01-27 03:00:00,1
7416745,R141,R031,00-00-00,34 ST-PENN STA,2022-01-27 07:00:00,1
7416746,R141,R031,00-00-00,34 ST-PENN STA,2022-01-27 11:00:00,1
7416747,R141,R031,00-00-00,34 ST-PENN STA,2022-01-27 15:00:00,1


No more duplicate Entries or Exits!!!

Let's clean up the data set and remove all columns we no longer need.

In [13]:
# Dropping Desc, Linename, and Division columns.
# To prevent errors in multiple run of cell, errors on drop is ignored
mta_df_clean = mta_df.drop(["DESC", "LINENAME", "DIVISION"], axis=1, errors="ignore")
mta_df_clean.head()


,C/A,UNIT,SCP,STATION,DATE,TIME,ENTRIES,EXITS,DATE_TIME
9441218,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/29/2022,21:00:00,0,217,2022-04-29 21:00:00
9441217,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/29/2022,17:00:00,0,215,2022-04-29 17:00:00
9441216,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/29/2022,13:00:00,0,215,2022-04-29 13:00:00
9441215,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/29/2022,09:00:00,0,215,2022-04-29 09:00:00
9441214,TRAM2,R469,00-05-01,RIT-ROOSEVELT,04/29/2022,05:00:00,0,214,2022-04-29 05:00:00


In [14]:
mta_df_clean.shape

(11125116, 9)

After cleaning dataset, we go from an original shape of <b>(40049393, 11)</b> to a current shape of <b>(11125116, 9)</b>. That's 28,924,277 ***LESS*** rows! This should hopefully improve processing speed.

Going to pickle this non-duplicate containing dataset for ease of future processing.

In [15]:
mta_df_clean.to_pickle("mta_df_clean.pkl")
#mta_df_clean = pd.read_pickle("mta_df_clean.pkl") 

### Calculating daily entries

Using Metis MTA Exercises 1-3 to assist.

In looking at the data, each row of data is how many entries/exists at a 4 hour timeslot at each station for each day. The `ENTRIES` column represents **cumulative entries** at that point in time.

In [16]:
turnstiles_daily = (mta_df_clean.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .ENTRIES.first())
turnstiles_daily.head(20)


,C/A,UNIT,SCP,STATION,DATE,ENTRIES
0,A002,R051,02-00-00,59 ST,01/01/2022,7675936
1,A002,R051,02-00-00,59 ST,01/02/2022,7676054
2,A002,R051,02-00-00,59 ST,01/03/2022,7676298
3,A002,R051,02-00-00,59 ST,01/04/2022,7676554
4,A002,R051,02-00-00,59 ST,01/05/2022,7676817
5,A002,R051,02-00-00,59 ST,01/06/2022,7677090
6,A002,R051,02-00-00,59 ST,01/07/2022,7677343
7,A002,R051,02-00-00,59 ST,01/08/2022,7677516
8,A002,R051,02-00-00,59 ST,01/09/2022,7677647
9,A002,R051,02-00-00,59 ST,01/10/2022,7677895


Calculating **daily entries**, i.e. the number of new entries gained each day.

In [17]:
turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "ENTRIES"]
                                                       .transform(lambda x: x.shift(1)))
# transform() takes a function as parameter
# shift moves the index by the number of periods given (positive or negative)

<ipython-input-17-7b8d2b8f5e46>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_daily[["PREV_DATE", "PREV_ENTRIES"]] = (turnstiles_daily


In [18]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
0,A002,R051,02-00-00,59 ST,01/01/2022,7675936,NaN,NaN
1,A002,R051,02-00-00,59 ST,01/02/2022,7676054,01/01/2022,7675936.0
2,A002,R051,02-00-00,59 ST,01/03/2022,7676298,01/02/2022,7676054.0
3,A002,R051,02-00-00,59 ST,01/04/2022,7676554,01/03/2022,7676298.0
4,A002,R051,02-00-00,59 ST,01/05/2022,7676817,01/04/2022,7676554.0


In [19]:
#The first date does not have a previous date so we will want to drop that date.

# Drop the rows for first date
turnstiles_daily.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
# axis = 0 means index (=1 means column)

In [20]:
#Let's check that the number of entries for today is higher than entries for yesterday

turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]].head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES
113,A002,R051,02-00-00,59 ST,04/24/2021,7561578,04/23/2022,7703881.0
115,A002,R051,02-00-00,59 ST,04/25/2021,7561843,04/24/2022,7704022.0
117,A002,R051,02-00-00,59 ST,04/26/2021,7562300,04/25/2022,7704364.0
119,A002,R051,02-00-00,59 ST,04/27/2021,7562873,04/26/2022,7704764.0
121,A002,R051,02-00-00,59 ST,04/28/2021,7563448,04/27/2022,7705198.0


In [21]:
#Let's see how many stations have this problem and how often.

(turnstiles_daily[turnstiles_daily["ENTRIES"] < turnstiles_daily["PREV_ENTRIES"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size()) # size() behaves same as if we'd done .DATE.count()

C/A    UNIT  SCP       STATION      
A002   R051  02-00-00  59 ST            7
             02-00-01  59 ST            7
             02-03-00  59 ST            7
             02-03-01  59 ST            7
             02-03-02  59 ST            7
                                       ..
TRAM2  R469  00-00-01  RIT-ROOSEVELT    7
             00-03-00  RIT-ROOSEVELT    7
             00-03-01  RIT-ROOSEVELT    7
             00-05-00  RIT-ROOSEVELT    8
             00-05-01  RIT-ROOSEVELT    7
Length: 4810, dtype: int64

In [22]:
#Let's clean these up.
#If the entries are smaller the day before let's make the sum positive. In order to avoid outliers let's cap the total daily count
#for daily entries to be 1,000,000 people.

def get_daily_counts(row, max_counter):
    counter = row["ENTRIES"] - row["PREV_ENTRIES"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["ENTRIES"], row["PREV_ENTRIES"])
        return 0
    return counter

# If counter is > 1 million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
turnstiles_daily["DAILY_ENTRIES"] = turnstiles_daily.apply(get_daily_counts, axis=1, max_counter=1000000) 

# apply takes a function as parameter and applies it along the given axis (1=apply by row) 
# apply takes care of passing each row to the function

1390089 148715.0
149117 1390089.0
1390113 149117.0
149568 1390113.0
1390141 149568.0
150123 1390141.0
1390167 150123.0
150717 1390167.0
1390171 150717.0
151298 1390171.0
1390173 151298.0
151845 1390173.0
1390173 151845.0
3082 1390173.0
5259320 54958.0
55150 5259320.0
5259443 55150.0
55421 5259443.0
5259630 55421.0
55731 5259630.0
5259832 55731.0
56066 5259832.0
5260017 56066.0
56397 5260017.0
5260206 56397.0
56718 5260206.0
5260410 56718.0
50 5264647.0
4336963 69410.0
69713 4336963.0
4337109 69713.0
70120 4337109.0
4337292 70120.0
70601 4337292.0
4337481 70601.0
71103 4337481.0
4337671 71103.0
71607 4337671.0
4337856 71607.0
72135 4337856.0
4338048 72135.0
18 4340953.0
846 7354911.0
7007307 121440.0
122296 7007307.0
7007948 122296.0
123760 7007948.0
7008928 123760.0
125401 7008928.0
7009952 125401.0
127144 7009952.0
7011164 127144.0
128873 7011164.0
7012295 128873.0
130495 7012295.0
7013487 130495.0
5564568 15719.0
15775 5564568.0
5564643 15775.0
15905 5564643.0
5564769 15905.0
16082 5

117440512 0.0
0 117440512.0
117440512 0.0
0 117440512.0
117440512 0.0
0 117440512.0
117440512 0.0
0 117440512.0
117440512 0.0
0 117440512.0
117440512 0.0
0 117440512.0
117440512 0.0
0 117440512.0
8875379 231485.0
231883 8875379.0
8875656 231883.0
232435 8875656.0
8876178 232435.0
233108 8876178.0
8876753 233108.0
233869 8876753.0
8877363 233869.0
234570 8877363.0
8877446 234570.0
235216 8877446.0
8877446 235216.0
135 8882903.0
4454729 213356.0
213714 4454729.0
7643335 251896.0
252341 7643335.0
176807 67261528.0
67262042 176807.0
177119 67262042.0
67262955 177119.0
177638 67262955.0
67264003 177638.0
178195 67264003.0
67265082 178195.0
178784 67265082.0
67266169 178784.0
179372 67266169.0
67267088 179372.0
179989 67267088.0
67136025 290390.0
119 5962569.0
5762840 69894.0
70414 5762840.0
5763167 70414.0
71292 5763167.0
5763687 71292.0
72175 5763687.0
5764267 72175.0
73233 5764267.0
5764926 73233.0
74297 5764926.0
5765530 74297.0
75311 5765530.0
5766118 75311.0
8928912 1979503446.0
197950

5 1940653.0
1913246 13268.0
13308 1913246.0
1913279 13308.0
13453 1913279.0
1913392 13453.0
13603 1913392.0
1913491 13603.0
13754 1913491.0
1913588 13754.0
13905 1913588.0
1913701 13905.0
14039 1913701.0
1913793 14039.0
2392699 495174.0
495231 2392699.0
2392756 495231.0
495404 2392756.0
2392912 495404.0
495572 2392912.0
2393040 495572.0
495730 2393040.0
2393180 495730.0
495883 2393180.0
2393328 495883.0
496048 2393328.0
67436550 375578.0
240 67470015.0
4068273 60894.0
60977 4068273.0
4068302 60977.0
61140 4068302.0
4068402 61140.0
61314 4068402.0
4068502 61314.0
61503 4068502.0
4068623 61503.0
61713 4068623.0
4068738 61713.0
61901 4068738.0
4068844 61901.0
143 4070964.0
58296876 123832.0
124027 58296876.0
58296961 124027.0
124425 58296961.0
58297212 124425.0
124844 58297212.0
58297482 124844.0
125305 58297482.0
58297779 125305.0
125753 58297779.0
58298077 125753.0
126183 58298077.0
58298356 126183.0
5 58304715.0
1043926553 2368212.0
2151581 1043925813.0
301 1191891.0
6169566 117164.0
1

1048912 9753966.0
9663741 1078127.0
1078196 9663741.0
9663789 1078196.0
1078670 9663789.0
9664056 1078670.0
1079166 9664056.0
9664392 1079166.0
1079675 9664392.0
9664733 1079675.0
1080198 9664733.0
9665073 1080198.0
1080664 9665073.0
9665385 1080664.0
85 3970147.0
3746869 34973.0
35497 3746869.0
3747619 35497.0
36260 3747619.0
3748671 36260.0
37032 3748671.0
3749713 37032.0
37741 3749713.0
3750328 37741.0
38512 3750328.0
3750882 38512.0
39216 3750882.0
3751552 39216.0
44 1189272.0
1164806 1879.0
1879 1164806.0
1164850 1879.0
1939 1164850.0
1164925 1939.0
2004 1164925.0
1164991 2004.0
2109 1164991.0
1165060 2109.0
2187 1165060.0
1165109 2187.0
2264 1165109.0
1165189 2264.0
2318641 71703.0
71703 2318641.0
2318738 71703.0
71917 2318738.0
2318858 71917.0
72202 2318858.0
2318943 72202.0
72480 2318943.0
2319095 72480.0
72749 2319095.0
2319210 72749.0
73039 2319210.0
2319361 73039.0
77 2320032.0
1840496 375158.0
376163 1840496.0
1841157 376163.0
377559 1841157.0
1842309 377559.0
379194 184230

136876645 545881.0
546403 136876645.0
136876936 546403.0
547120 136876936.0
136877520 547120.0
548014 136877520.0
136878149 548014.0
548999 136878149.0
136878833 548999.0
549980 136878833.0
136879491 549980.0
550906 136879491.0
136880139 550906.0
393940 136995130.0
3901604 115703.0
116197 3901604.0
3902116 116197.0
116928 3902116.0
3902787 116928.0
117638 3902787.0
3903539 117638.0
118382 3903539.0
3904291 118382.0
119195 3904291.0
3905061 119195.0
119996 3905061.0
3905900 119996.0
443 4026667.0
1996003 86679.0
86903 1996003.0
1996141 86903.0
87351 1996141.0
1996383 87351.0
87844 1996383.0
1996629 87844.0
88343 1996629.0
1996880 88343.0
88778 1996880.0
1997179 88778.0
89270 1997179.0
1997503 89270.0
124 2030205.0
7663237 53520.0
53564 7663237.0
7663288 53564.0
53964 7663288.0
7663454 53964.0
54352 7663454.0
7663618 54352.0
54753 7663618.0
7663775 54753.0
55197 7663775.0
7663925 55197.0
55513 7663925.0
7664040 55513.0
112 7682427.0
12895988 1588577.0
1588777 12895988.0
12896200 1588777.

In [23]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
1,A002,R051,02-00-00,59 ST,01/02/2022,7676054,01/01/2022,7675936.0,118.0
2,A002,R051,02-00-00,59 ST,01/03/2022,7676298,01/02/2022,7676054.0,244.0
3,A002,R051,02-00-00,59 ST,01/04/2022,7676554,01/03/2022,7676298.0,256.0
4,A002,R051,02-00-00,59 ST,01/05/2022,7676817,01/04/2022,7676554.0,263.0
5,A002,R051,02-00-00,59 ST,01/06/2022,7677090,01/05/2022,7676817.0,273.0


In [24]:
turnstiles_daily.to_pickle("turnstiles_daily.pkl")
#turnstiles_daily = pd.read_pickle("turnstiles_daily.pkl")  

### Calculating daily exits
* Using same methodology to calculate `EXITS` as I did for `ENTRIES`.

In [25]:
turnstiles_daily_exits = (mta_df_clean.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"],as_index=False)
                        .EXITS.first())
turnstiles_daily_exits.head(20)


,C/A,UNIT,SCP,STATION,DATE,EXITS
0,A002,R051,02-00-00,59 ST,01/01/2022,2649668
1,A002,R051,02-00-00,59 ST,01/02/2022,2649829
2,A002,R051,02-00-00,59 ST,01/03/2022,2650233
3,A002,R051,02-00-00,59 ST,01/04/2022,2650658
4,A002,R051,02-00-00,59 ST,01/05/2022,2651066
5,A002,R051,02-00-00,59 ST,01/06/2022,2651526
6,A002,R051,02-00-00,59 ST,01/07/2022,2651918
7,A002,R051,02-00-00,59 ST,01/08/2022,2652151
8,A002,R051,02-00-00,59 ST,01/09/2022,2652319
9,A002,R051,02-00-00,59 ST,01/10/2022,2652756


- Work off of the daily maximum `EXITS` calculations from above. Recall that the `EXITS` column contains **cumulative exits** on each day. We would now like you to calculate **daily exits**, i.e. the number of new exits gained each day.
- Hint: Group the data by turnstile and use the Pandas `.apply()` method to compute the same differencing function for each turnstile. Check out the `.shift()` and `.diff()` DataFrame methods for this purpose.

In [26]:
turnstiles_daily_exits[["PREV_DATE", "PREV_EXITS"]] = (turnstiles_daily_exits
                                                       .groupby(["C/A", "UNIT", "SCP", "STATION"])["DATE", "EXITS"]
                                                       .transform(lambda x: x.shift(1)))
# transform() takes a function as parameter
# shift moves the index by the number of periods given (positive or negative)

<ipython-input-26-155a9950ea0b>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  turnstiles_daily_exits[["PREV_DATE", "PREV_EXITS"]] = (turnstiles_daily_exits


In [27]:
turnstiles_daily_exits.head()

,C/A,UNIT,SCP,STATION,DATE,EXITS,PREV_DATE,PREV_EXITS
0,A002,R051,02-00-00,59 ST,01/01/2022,2649668,NaN,NaN
1,A002,R051,02-00-00,59 ST,01/02/2022,2649829,01/01/2022,2649668.0
2,A002,R051,02-00-00,59 ST,01/03/2022,2650233,01/02/2022,2649829.0
3,A002,R051,02-00-00,59 ST,01/04/2022,2650658,01/03/2022,2650233.0
4,A002,R051,02-00-00,59 ST,01/05/2022,2651066,01/04/2022,2650658.0


In [28]:
#The first date does not have a previous date so we will want to drop that date.

# Drop the rows for first date
turnstiles_daily_exits.dropna(subset=["PREV_DATE"], axis=0, inplace=True)
# axis = 0 means index (=1 means column)

In [29]:
#Let's check that the number of exits for today is higher than entries for yesterday

turnstiles_daily_exits[turnstiles_daily_exits["EXITS"] < turnstiles_daily_exits["PREV_EXITS"]].head()


,C/A,UNIT,SCP,STATION,DATE,EXITS,PREV_DATE,PREV_EXITS
113,A002,R051,02-00-00,59 ST,04/24/2021,2583031,04/23/2022,2696011.0
115,A002,R051,02-00-00,59 ST,04/25/2021,2583132,04/24/2022,2696197.0
117,A002,R051,02-00-00,59 ST,04/26/2021,2583332,04/25/2022,2696739.0
119,A002,R051,02-00-00,59 ST,04/27/2021,2583535,04/26/2022,2697237.0
121,A002,R051,02-00-00,59 ST,04/28/2021,2583805,04/27/2022,2697739.0


In [30]:
#Let's see how many stations have this problem and how often.

(turnstiles_daily_exits[turnstiles_daily_exits["EXITS"] < turnstiles_daily_exits["PREV_EXITS"]]
    .groupby(["C/A", "UNIT", "SCP", "STATION"])
    .size()) # size() behaves same as if we'd done .DATE.count()

C/A    UNIT  SCP       STATION      
A002   R051  02-00-00  59 ST            7
             02-00-01  59 ST            7
             02-03-00  59 ST            7
             02-03-01  59 ST            7
             02-03-02  59 ST            7
                                       ..
TRAM2  R469  00-00-01  RIT-ROOSEVELT    7
             00-03-00  RIT-ROOSEVELT    7
             00-03-01  RIT-ROOSEVELT    7
             00-05-00  RIT-ROOSEVELT    8
             00-05-01  RIT-ROOSEVELT    7
Length: 4803, dtype: int64

In [31]:
#Let's clean these up.
#If the entries are smaller the day before let's make the sum positive. In order to avoid outliers let's cap the total daily count
#for daily entries to be 1,000,000 people.

def get_daily_EXIT_counts(row, max_counter):
    counter = row["EXITS"] - row["PREV_EXITS"]
    if counter < 0:
        counter = -counter
    if counter > max_counter:
        print(row["EXITS"], row["PREV_EXITS"])
        return 0
    return counter

# If counter is > 1 million, then the counter might have been reset.  
# Just set it to zero as different counters have different cycle limits
turnstiles_daily_exits["DAILY_EXITS"] = turnstiles_daily_exits.apply(get_daily_EXIT_counts, axis=1, max_counter=1000000) 

# apply takes a function as parameter and applies it along the given axis (1=apply by row) 
# apply takes care of passing each row to the function

932 1133982.0
1134325 932.0
5973125 129245.0
129795 5973125.0
5973382 129795.0
130447 5973382.0
5973608 130447.0
131328 5973608.0
5973819 131328.0
132175 5973819.0
5974064 132175.0
133007 5974064.0
5974317 133007.0
133735 5974317.0
5974614 133735.0
85 5980618.0
93153 1153244.0
1155190 93153.0
93774 1155190.0
1158388 93774.0
95822 1158388.0
1162108 95822.0
98163 1162108.0
1166039 98163.0
100568 1166039.0
1169976 100568.0
103012 1169976.0
1174121 103012.0
105581 1174121.0
33119044 34157641.0
34159454 33119044.0
33119900 34159454.0
34162848 33119900.0
33121933 34162848.0
34166828 33121933.0
33124288 34166828.0
34170890 33124288.0
33126632 34170890.0
34174834 33126632.0
33129070 34174834.0
34178508 33129070.0
33131511 34178508.0
589 5976643.0
5523895 187381.0
188754 5523895.0
5524708 188754.0
190940 5524708.0
5525984 190940.0
193282 5525984.0
5527384 193282.0
195996 5527384.0
5528867 195996.0
198524 5528867.0
5530299 198524.0
200908 5530299.0
5531750 200908.0
17921224 19012764.0
19015634 1

3602317 4807944.0
4810698 3602317.0
3603965 4810698.0
4814435 3603965.0
3606199 4814435.0
4818587 3606199.0
3608438 4818587.0
4823206 3608438.0
3610754 4823206.0
4827716 3610754.0
3613223 4827716.0
4832258 3613223.0
3615837 4832258.0
1919 1435144.0
306 1150397.0
27655491 29116374.0
29120040 27655491.0
27657458 29120040.0
29124120 27657458.0
27659858 29124120.0
29128877 27659858.0
27662469 29128877.0
29134179 27662469.0
27665662 29134179.0
29139424 27665662.0
27668383 29139424.0
29144960 27668383.0
27671681 29144960.0
586804 1680436.0
1683788 586804.0
588573 1683788.0
1686427 588573.0
590198 1686427.0
1689462 590198.0
592001 1689462.0
1693026 592001.0
594196 1693026.0
1696454 594196.0
596036 1696454.0
1700298 596036.0
598477 1700298.0
2133796536 55608.0
55645 2133796536.0
2774389 332209.0
333050 2774389.0
6592830 122133.0
122476 6592830.0
84117 16892466.0
16892711 84117.0
84202 16892711.0
16893166 84202.0
84445 16893166.0
16893669 84445.0
84716 16893669.0
16894207 84716.0
84991 16894207

3076202 92844.0
93075 3076202.0
3076388 93075.0
93542 3076388.0
3076752 93542.0
93983 3076752.0
3077089 93983.0
94463 3077089.0
3077416 94463.0
94942 3077416.0
3077761 94942.0
95366 3077761.0
5007350 94537.0
94657 5007350.0
5007386 94657.0
94926 5007386.0
5007540 94926.0
95186 5007540.0
5007731 95186.0
95649 5007731.0
5007889 95649.0
95986 5007889.0
5008067 95986.0
96329 5008067.0
5008217 96329.0
189 5011886.0
11474337 244047.0
244395 11474337.0
11474467 244395.0
245172 11474467.0
11475047 245172.0
246136 11475047.0
11475627 246136.0
247339 11475627.0
11476248 247339.0
248291 11476248.0
11476837 248291.0
249142 11476837.0
11477352 249142.0
0 11489500.0
1009319248 1267624.0
1039140 1009318796.0
2391015 3502732.0
3505773 2391015.0
2393067 3505773.0
3508989 2393067.0
2395581 3508989.0
3512852 2395581.0
2397982 3512852.0
3515893 2397982.0
2400618 3515893.0
3519523 2400618.0
2403164 3519523.0
3523119 2403164.0
2406007 3523119.0
59234824 60307119.0
60308808 59234824.0
59235350 60308808.0
603

2368964 604002048.0
604002048 2368964.0
2368964 604002048.0
604002048 2368964.0
2368964 604002048.0
604002048 2368964.0
2368964 604002048.0
604002048 2368964.0
2368964 604002048.0
604002048 2368964.0
2368964 604002048.0
604002057 2368964.0
2368974 604002057.0
604001837 2369108.0
144 11036345.0
10904048 47487.0
47608 10904048.0
10904156 47608.0
48391 10904156.0
10904609 48391.0
49268 10904609.0
10905068 49268.0
50089 10905068.0
10905532 50089.0
50962 10905532.0
10906050 50962.0
51747 10906050.0
10906500 51747.0
331 3944385.0
3764245 15641.0
15641 3764245.0
3764482 15641.0
16363 3764482.0
3764851 16363.0
17190 3764851.0
3765276 17190.0
18057 3765276.0
3765756 18057.0
18990 3765756.0
3766244 18990.0
19883 3766244.0
3766754 19883.0
114 1560263.0
1475463 10108.0
10108 1475463.0
1475614 10108.0
10399 1475614.0
1475777 10399.0
10775 1475777.0
1475962 10775.0
11151 1475962.0
1476143 11151.0
11549 1476143.0
1476342 11549.0
11919 1476342.0
1476563 11919.0
6660027 439610.0
439610 6660027.0
666061

7535247 286293.0
287488 7535247.0
7535882 287488.0
289265 7535882.0
7537099 289265.0
291357 7537099.0
7538473 291357.0
293639 7538473.0
7539973 293639.0
295765 7539973.0
7541460 295765.0
297818 7541460.0
7542877 297818.0
891 7766949.0
1986 1058913.0
5175953 69611.0
69692 5175953.0
5175983 69692.0
70304 5175983.0
5176110 70304.0
70845 5176110.0
5176252 70845.0
71504 5176252.0
5176382 71504.0
72145 5176382.0
5176489 72145.0
72633 5176489.0
5176585 72633.0
32 5192355.0
9172899 352904.0
353865 9172899.0
9173882 353865.0
355266 9173882.0
9175356 355266.0
356723 9175356.0
9177115 356723.0
358247 9177115.0
9178795 358247.0
359717 9178795.0
9180564 359717.0
361231 9180564.0
9182381 361231.0
440 9252156.0
187 1015906.0
6714542 2117913.0
4860815 40122.0
40380 4860815.0
4860977 40380.0
40766 4860977.0
4861248 40766.0
41154 4861248.0
4861492 41154.0
41552 4861492.0
4861781 41552.0
41950 4861781.0
4862076 41950.0
42347 4862076.0
4862342 42347.0
135 4935826.0
10879546 280818.0
281429 10879546.0
2458

In [32]:
turnstiles_daily_exits.head()

,C/A,UNIT,SCP,STATION,DATE,EXITS,PREV_DATE,PREV_EXITS,DAILY_EXITS
1,A002,R051,02-00-00,59 ST,01/02/2022,2649829,01/01/2022,2649668.0,161.0
2,A002,R051,02-00-00,59 ST,01/03/2022,2650233,01/02/2022,2649829.0,404.0
3,A002,R051,02-00-00,59 ST,01/04/2022,2650658,01/03/2022,2650233.0,425.0
4,A002,R051,02-00-00,59 ST,01/05/2022,2651066,01/04/2022,2650658.0,408.0
5,A002,R051,02-00-00,59 ST,01/06/2022,2651526,01/05/2022,2651066.0,460.0


In [33]:
turnstiles_daily_exits.to_pickle("turnstiles_daily_exits.pkl")
#turnstiles_daily_exits = pd.read_pickle("turnstiles_daily_exits.pkl")

Going to ***merge/join*** turnstiles_daily_exits dataframe with turnstiles_daily dataframe.
* Need to first rename some columns

In [34]:
turnstiles_daily_exits.rename(columns = {'PREV_DATE':'PREV_DATE_EXITS'}, inplace = True)
turnstiles_daily_exits.head()

,C/A,UNIT,SCP,STATION,DATE,EXITS,PREV_DATE_EXITS,PREV_EXITS,DAILY_EXITS
1,A002,R051,02-00-00,59 ST,01/02/2022,2649829,01/01/2022,2649668.0,161.0
2,A002,R051,02-00-00,59 ST,01/03/2022,2650233,01/02/2022,2649829.0,404.0
3,A002,R051,02-00-00,59 ST,01/04/2022,2650658,01/03/2022,2650233.0,425.0
4,A002,R051,02-00-00,59 ST,01/05/2022,2651066,01/04/2022,2650658.0,408.0
5,A002,R051,02-00-00,59 ST,01/06/2022,2651526,01/05/2022,2651066.0,460.0


In [35]:
turnstiles_daily.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES
1,A002,R051,02-00-00,59 ST,01/02/2022,7676054,01/01/2022,7675936.0,118.0
2,A002,R051,02-00-00,59 ST,01/03/2022,7676298,01/02/2022,7676054.0,244.0
3,A002,R051,02-00-00,59 ST,01/04/2022,7676554,01/03/2022,7676298.0,256.0
4,A002,R051,02-00-00,59 ST,01/05/2022,7676817,01/04/2022,7676554.0,263.0
5,A002,R051,02-00-00,59 ST,01/06/2022,7677090,01/05/2022,7676817.0,273.0


In [36]:
total_turnstile_traffic = pd.merge(turnstiles_daily, turnstiles_daily_exits, on=["C/A", "UNIT", "SCP", "STATION", "DATE"])


In [37]:
total_turnstile_traffic.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES,EXITS,PREV_DATE_EXITS,PREV_EXITS,DAILY_EXITS
0,A002,R051,02-00-00,59 ST,01/02/2022,7676054,01/01/2022,7675936.0,118.0,2649829,01/01/2022,2649668.0,161.0
1,A002,R051,02-00-00,59 ST,01/03/2022,7676298,01/02/2022,7676054.0,244.0,2650233,01/02/2022,2649829.0,404.0
2,A002,R051,02-00-00,59 ST,01/04/2022,7676554,01/03/2022,7676298.0,256.0,2650658,01/03/2022,2650233.0,425.0
3,A002,R051,02-00-00,59 ST,01/05/2022,7676817,01/04/2022,7676554.0,263.0,2651066,01/04/2022,2650658.0,408.0
4,A002,R051,02-00-00,59 ST,01/06/2022,7677090,01/05/2022,7676817.0,273.0,2651526,01/05/2022,2651066.0,460.0


This dataset has the daily `ENTRIES` and `EXITS` which have been cleaned similarly to the method above. The total traffic was calculated from the sum of the entries and exits. We can use this dataset to find the highest trafficked subway station. Below we see that Penn Station takes that mark followed by Grand Central Station.

In [38]:
total_turnstile_traffic["total_traffic"] = total_turnstile_traffic.DAILY_ENTRIES + total_turnstile_traffic.DAILY_EXITS


In [39]:
total_turnstile_traffic.head()

,C/A,UNIT,SCP,STATION,DATE,ENTRIES,PREV_DATE,PREV_ENTRIES,DAILY_ENTRIES,EXITS,PREV_DATE_EXITS,PREV_EXITS,DAILY_EXITS,total_traffic
0,A002,R051,02-00-00,59 ST,01/02/2022,7676054,01/01/2022,7675936.0,118.0,2649829,01/01/2022,2649668.0,161.0,279.0
1,A002,R051,02-00-00,59 ST,01/03/2022,7676298,01/02/2022,7676054.0,244.0,2650233,01/02/2022,2649829.0,404.0,648.0
2,A002,R051,02-00-00,59 ST,01/04/2022,7676554,01/03/2022,7676298.0,256.0,2650658,01/03/2022,2650233.0,425.0,681.0
3,A002,R051,02-00-00,59 ST,01/05/2022,7676817,01/04/2022,7676554.0,263.0,2651066,01/04/2022,2650658.0,408.0,671.0
4,A002,R051,02-00-00,59 ST,01/06/2022,7677090,01/05/2022,7676817.0,273.0,2651526,01/05/2022,2651066.0,460.0,733.0


In [40]:
total_turnstile_traffic.to_pickle("total_turnstile_traffic.pkl")
#total_turnstile_traffic = pd.read_pickle("total_turnstile_traffic.pkl")

In [41]:
total_station_traffic = pd.DataFrame(total_turnstile_traffic.groupby(['STATION'])['total_traffic'].sum().sort_values(ascending = False).reset_index())
total_station_traffic.head(10)


,STATION,total_traffic
0,34 ST-PENN STA,538678766.0
1,GRD CNTRL-42 ST,417873985.0
2,23 ST,378164910.0
3,34 ST-HERALD SQ,355704602.0
4,86 ST,341824561.0
5,125 ST,325917900.0
6,14 ST-UNION SQ,322191333.0
7,FULTON ST,318168531.0
8,TIMES SQ-42 ST,312185285.0
9,42 ST-PORT AUTH,302571269.0
